In [1]:
#包文件
import numpy as np
import matplotlib.pyplot as plt
import random
from sympy import *
import math 
from datetime import datetime

In [2]:
#定义变量 
agent_resource_quantity = np.zeros(shape=(54,2000), dtype=np.float64) #100个agent的资源量 定义成一个矩阵
sum_resource=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户资源量的总和
sum_cost=np.zeros(shape=(1,2000), dtype=np.float64)#存放每一次迭代后所有用户成本的总和
iteration_update = 1 #迭代步长 
A=np.loadtxt("Adjacency_Matrix_generator_zero.txt") #网络拓扑的邻接矩阵 对角元素要为0
D=np.zeros(shape=(54,54))# 网络的度矩阵
L=np.zeros(shape=(54,54))# 网络的拉普拉斯矩阵 L=D-A
W=np.zeros(shape=(54,54)) #可以直接采用网络对应的拉普拉斯矩阵  因为无向图的拉普拉斯矩阵天然满足行和和列和为0
iteration=[]#绘图横坐标
generator_a_values= np.loadtxt("generator_a_values.txt")#每个agent的成本函数中的超参数a
generator_b_values= np.loadtxt("generator_b_values.txt")#每个agent的成本函数中的超参数a
generator_c_values= np.loadtxt("generator_c_values.txt")#每个agent的成本函数中的超参数a
agent_gradient=np.zeros(shape=(54,2000), dtype=np.float64) 
bias=np.loadtxt("2000_Gaussian_distribution_(80,10).txt")
C=np.zeros(shape=(1,2000), dtype=np.float64)#adaptive robust clipping的裁剪参数 
#设置为全局第b大的梯度信息 b为网络中设备损坏的用户上界 每一次迭代就需要计算一次新的C值

In [3]:
#根据邻接矩阵生成网络对应的度矩阵
for i in range(0,54):
    d=0
    for j in range(0,54):
        if A[i][j]==1:
            d=d+1
    D[i][i]=d

#根据邻接矩阵和度矩阵计算拉普拉斯矩阵
L=D-A

#选择用拉普拉斯矩阵作为聚合矩阵
W=L
    
#资源量的初始化 需要给定可行的初始化 即满足平均资源约束
for i in range (0,54):
    agent_resource_quantity[i][0]=6000/54
    
#总资源量 初始化
sum=0
for i in range (0,54):
    sum=sum+agent_resource_quantity[i][0]
sum_resource[0]=sum

iteration.append(0)

#初始梯度计算 随机选择一个用户作为biased 用户
for i in range (0,54):
    if i==48:
        agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]+bias[0]
    else:
        agent_gradient[i][0]=2*generator_c_values[i]*agent_resource_quantity[i][0]+generator_b_values[i]
        
#用户总成本初始化
sum=0
for i in range(0,54):
    sum=sum+(generator_c_values[i]*(agent_resource_quantity[i][0])*(agent_resource_quantity[i][0])+generator_b_values[i]*agent_resource_quantity[i][0]+generator_a_values[i])
sum_cost[0][0]=sum
    
#计算初始的C值 因为网络中噪声用户的数量为1 所以选择第4大的梯度作为clipping的阈值
column_data = agent_gradient[:, 0]
second_largest = np.partition(np.abs(column_data), -2)[-2]
C[0][0]=second_largest

In [4]:
#基于初始化的权重梯度下降无中心资源分配算法
def IWDGD_Gaussian_bias_C(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias,C):
    
    #显示初始化的数据
    #输出初始状态的原变量和对偶变量
    print("****************************IWDGD算法+计算误差+鲁棒****************************")
    print("相关变量初始值为：")
    for i in range (0,54):
        print("agent",(i+1),"的初始资源申请量为:",agent_resource_quantity[i][0])
    print("*************************")
    print("初始资源申请量总和为:",sum_resource[0][0])
    print("*************************")
    print("初始成本总和为:",sum_cost[0][0])
    print("*************************")
    print("迭代步长为:",iteration_update)
    print("*****************************************************************************")
    
    #开始迭代
    for k in range(1,2000):#迭代次数
        iteration.append(k)
 
        #所有用户资源申请量基于可行初始化权重梯度下降的迭代  假设部分用户因为设备损坏 导致梯度的计算有误差     
        for i in range (0,54):
            weight_gradient_sum = np.float64(0)
            for j in range (0,54):  
                if np.abs(agent_gradient[j][k-1])>C[0][k-1]:
                    weight_gradient_sum=weight_gradient_sum+W[i][j]*(C[0][k-1]/np.abs(agent_gradient[j][k-1])*agent_gradient[j][k-1])
                else:
                    weight_gradient_sum=weight_gradient_sum+W[i][j]*agent_gradient[j][k-1]                   
                #weight_gradient_sum = np.round(weight_gradient_sum, decimals=3)
            agent_resource_quantity[i][k]=agent_resource_quantity[i][k-1]-iteration_update*weight_gradient_sum
                #agent_resource_quantity[i][k] = np.round(agent_resource_quantity[i][k], decimals=3)                      
        
        #计算所有用户的资源量总和
        sum1=np.float64(0)
        for i in range (0,54):
            sum1 = sum1 + agent_resource_quantity[i][k]
        sum_resource[0][k]=sum1
        
        #计算所有用户的成本总和
        sum2=np.float64(0)
        for i in range(0,54):
            sum2=sum2+(generator_c_values[i]*(agent_resource_quantity[i][k])*(agent_resource_quantity[i][k])+generator_b_values[i]*agent_resource_quantity[i][k]+generator_a_values[i])
        sum_cost[0][k]=sum2
        
        #计算当前迭代的用户梯度
        for i in range (0,54): 
            if i==48:
                agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]+bias[k]
            else:
                agent_gradient[i][k]=2*generator_c_values[i]*agent_resource_quantity[i][k]+generator_b_values[i]
                
        #计算当前迭代的C值
        column_data = agent_gradient[:, k]
        second_largest = np.partition(np.abs(column_data), -2)[-2]
        C[0][k]=second_largest
        
        #输出第k次迭代中的结果 每间隔2000次迭代输出一次结果
        if (k+1)%20==0:
        #if (k+1)%2000==0:
            print(k,"次迭代中的结果：")
            for i in range (0,54):
                print("agent",(i+1),"在",k,"次迭代中的资源申请量为:",agent_resource_quantity[i][k])
            print("资源申请量总和为:",sum_resource[0][k])
            print("****************************************************************************")
            print("成本总和为:",sum_cost[0][k])
            print("****************************************************************************")
            print("迭代步长为:",iteration_update)
            print("****************************************************************************")
        
        #iteration_update=1/(pow(k+1,0.05)) #递减步长
        
    print(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3])
    
    return None

In [5]:
IWDGD_Gaussian_bias_C(generator_a_values,generator_b_values,generator_c_values,agent_resource_quantity,sum_resource,iteration_update,W,iteration,bias,C)

****************************IWDGD算法+计算误差+鲁棒****************************
相关变量初始值为：
agent 1 的初始资源申请量为: 111.11111111111111
agent 2 的初始资源申请量为: 111.11111111111111
agent 3 的初始资源申请量为: 111.11111111111111
agent 4 的初始资源申请量为: 111.11111111111111
agent 5 的初始资源申请量为: 111.11111111111111
agent 6 的初始资源申请量为: 111.11111111111111
agent 7 的初始资源申请量为: 111.11111111111111
agent 8 的初始资源申请量为: 111.11111111111111
agent 9 的初始资源申请量为: 111.11111111111111
agent 10 的初始资源申请量为: 111.11111111111111
agent 11 的初始资源申请量为: 111.11111111111111
agent 12 的初始资源申请量为: 111.11111111111111
agent 13 的初始资源申请量为: 111.11111111111111
agent 14 的初始资源申请量为: 111.11111111111111
agent 15 的初始资源申请量为: 111.11111111111111
agent 16 的初始资源申请量为: 111.11111111111111
agent 17 的初始资源申请量为: 111.11111111111111
agent 18 的初始资源申请量为: 111.11111111111111
agent 19 的初始资源申请量为: 111.11111111111111
agent 20 的初始资源申请量为: 111.11111111111111
agent 21 的初始资源申请量为: 111.11111111111111
agent 22 的初始资源申请量为: 111.11111111111111
agent 23 的初始资源申请量为: 111.11111111111111
agent 24 的初始资源申请量为: 111.111111

49 次迭代中的结果：
agent 1 在 49 次迭代中的资源申请量为: 48.23689313596557
agent 2 在 49 次迭代中的资源申请量为: 60.77111873069868
agent 3 在 49 次迭代中的资源申请量为: 43.661965294812376
agent 4 在 49 次迭代中的资源申请量为: 64.25940958434225
agent 5 在 49 次迭代中的资源申请量为: 65.05940958434226
agent 6 在 49 次迭代中的资源申请量为: 71.80536807971976
agent 7 在 49 次迭代中的资源申请量为: 39.33965350690171
agent 8 在 49 次迭代中的资源申请量为: 37.602670925977144
agent 9 在 49 次迭代中的资源申请量为: 42.96196529481239
agent 10 在 49 次迭代中的资源申请量为: 55.596415054412525
agent 11 在 49 次迭代中的资源申请量为: 45.328337762356455
agent 12 在 49 次迭代中的资源申请量为: 49.13689313596558
agent 13 在 49 次迭代中的资源申请量为: 65.55940958434228
agent 14 在 49 次迭代中的资源申请量为: 54.99641505441252
agent 15 在 49 次迭代中的资源申请量为: 43.36196529481238
agent 16 在 49 次迭代中的资源申请量为: 43.56196529481238
agent 17 在 49 次迭代中的资源申请量为: 44.72833776235646
agent 18 在 49 次迭代中的资源申请量为: 46.32833776235645
agent 19 在 49 次迭代中的资源申请量为: 41.18074522146854
agent 20 在 49 次迭代中的资源申请量为: 55.49641505441253
agent 21 在 49 次迭代中的资源申请量为: 38.63965350690171
agent 22 在 49 次迭代中的资源申请量为: 38.93965350690172
age

89 次迭代中的结果：
agent 1 在 89 次迭代中的资源申请量为: 48.1478902277333
agent 2 在 89 次迭代中的资源申请量为: 60.659896456961086
agent 3 在 89 次迭代中的资源申请量为: 43.58344358385418
agent 4 在 89 次迭代中的资源申请量为: 64.13817496394701
agent 5 在 89 次迭代中的资源申请量为: 64.93817496394702
agent 6 在 89 次迭代中的资源申请量为: 71.67224382697499
agent 7 在 89 次迭代中的资源申请量为: 39.26940192645733
agent 8 在 89 次迭代中的资源申请量为: 37.535933194301315
agent 9 在 89 次迭代中的资源申请量为: 42.88344358385419
agent 10 在 89 次迭代中的资源申请量为: 55.493718100505006
agent 11 在 89 次迭代中的资源申请量为: 45.24490355411193
agent 12 在 89 次迭代中的资源申请量为: 49.04789022773331
agent 13 在 89 次迭代中的资源申请量为: 65.43817496394702
agent 14 在 89 次迭代中的资源申请量为: 54.893718100505005
agent 15 在 89 次迭代中的资源申请量为: 43.28344358385418
agent 16 在 89 次迭代中的资源申请量为: 43.48344358385418
agent 17 在 89 次迭代中的资源申请量为: 44.64490355411193
agent 18 在 89 次迭代中的资源申请量为: 46.24490355411193
agent 19 在 89 次迭代中的资源申请量为: 41.106588799047
agent 20 在 89 次迭代中的资源申请量为: 55.39371810050501
agent 21 在 89 次迭代中的资源申请量为: 38.569401926457324
agent 22 在 89 次迭代中的资源申请量为: 38.86940192645733
agent

109 次迭代中的结果：
agent 1 在 109 次迭代中的资源申请量为: 48.14793273366527
agent 2 在 109 次迭代中的资源申请量为: 60.659915917070634
agent 3 在 109 次迭代中的资源申请量为: 43.58347005911769
agent 4 在 109 次迭代中的资源申请量为: 64.1380900913388
agent 5 在 109 次迭代中的资源申请量为: 64.93809009133881
agent 6 在 109 次迭代中的资源申请量为: 71.67189910066391
agent 7 在 109 次迭代中的资源申请量为: 39.26942057921117
agent 8 在 109 次迭代中的资源申请量为: 37.53594955025081
agent 9 在 109 次迭代中的资源申请量为: 42.88347005911769
agent 10 在 109 次迭代中的资源申请量为: 55.49376853884032
agent 11 在 109 次迭代中的资源申请量为: 45.244936937811985
agent 12 在 109 次迭代中的资源申请量为: 49.04793273366528
agent 13 在 109 次迭代中的资源申请量为: 65.43809009133881
agent 14 在 109 次迭代中的资源申请量为: 54.89376853884033
agent 15 在 109 次迭代中的资源申请量为: 43.28347005911768
agent 16 在 109 次迭代中的资源申请量为: 43.483470059117685
agent 17 在 109 次迭代中的资源申请量为: 44.644936937811984
agent 18 在 109 次迭代中的资源申请量为: 46.244936937811985
agent 19 在 109 次迭代中的资源申请量为: 41.10661061138929
agent 20 在 109 次迭代中的资源申请量为: 55.393768538840334
agent 21 在 109 次迭代中的资源申请量为: 38.569420579211176
agent 22 在 109 次迭代中的资源申请

129 次迭代中的结果：
agent 1 在 129 次迭代中的资源申请量为: 48.14793273367247
agent 2 在 129 次迭代中的资源申请量为: 60.65991591709059
agent 3 在 129 次迭代中的资源申请量为: 43.583470059122774
agent 4 在 129 次迭代中的资源申请量为: 64.13809009137155
agent 5 在 129 次迭代中的资源申请量为: 64.93809009137155
agent 6 在 129 次迭代中的资源申请量为: 71.67189910050871
agent 7 在 129 次迭代中的资源申请量为: 39.26942057921511
agent 8 在 129 次迭代中的资源申请量为: 37.53594955025436
agent 9 在 129 次迭代中的资源申请量为: 42.88347005912277
agent 10 在 129 次迭代中的资源申请量为: 55.49376853885285
agent 11 在 129 次迭代中的资源申请量为: 45.24493693781794
agent 12 在 129 次迭代中的资源申请量为: 49.047932733672475
agent 13 在 129 次迭代中的资源申请量为: 65.43809009137155
agent 14 在 129 次迭代中的资源申请量为: 54.89376853885285
agent 15 在 129 次迭代中的资源申请量为: 43.28347005912277
agent 16 在 129 次迭代中的资源申请量为: 43.48347005912277
agent 17 在 129 次迭代中的资源申请量为: 44.644936937817945
agent 18 在 129 次迭代中的资源申请量为: 46.24493693781795
agent 19 在 129 次迭代中的资源申请量为: 41.10661061139373
agent 20 在 129 次迭代中的资源申请量为: 55.393768538852854
agent 21 在 129 次迭代中的资源申请量为: 38.56942057921511
agent 22 在 129 次迭代中的资源申请量为

...

1909 次迭代中的结果：
agent 1 在 1909 次迭代中的资源申请量为: 48.09018976891687
agent 2 在 1909 次迭代中的资源申请量为: 60.5877372111461
agent 3 在 1909 次迭代中的资源申请量为: 43.53252038433841
agent 4 在 1909 次迭代中的资源申请量为: 64.05934968488663
agent 5 在 1909 次迭代中的资源申请量为: 64.85934968488662
agent 6 在 1909 次迭代中的资源申请量为: 71.5852846533753
agent 7 在 1909 次迭代中的资源申请量为: 39.22383402809225
agent 8 在 1909 次迭代中的资源申请量为: 37.49264232668764
agent 9 在 1909 次迭代中的资源申请量为: 42.832520384338395
agent 10 在 1909 次迭代中的资源申请量为: 55.42714204105789
agent 11 在 1909 次迭代中的资源申请量为: 45.19080290835954
agent 12 在 1909 次迭代中的资源申请量为: 48.99018976891685
agent 13 在 1909 次迭代中的资源申请量为: 65.3593496848866
agent 14 在 1909 次迭代中的资源申请量为: 54.82714204105789
agent 15 在 1909 次迭代中的资源申请量为: 43.23252038433839
agent 16 在 1909 次迭代中的资源申请量为: 43.432520384338396
agent 17 在 1909 次迭代中的资源申请量为: 44.59080290835954
agent 18 在 1909 次迭代中的资源申请量为: 46.19080290835954
agent 19 在 1909 次迭代中的资源申请量为: 41.058491474097366
agent 20 在 1909 次迭代中的资源申请量为: 55.3271420410579
agent 21 在 1909 次迭代中的资源申请量为: 38.523834028092246
agent 22

1929 次迭代中的结果：
agent 1 在 1929 次迭代中的资源申请量为: 48.09018976891687
agent 2 在 1929 次迭代中的资源申请量为: 60.5877372111461
agent 3 在 1929 次迭代中的资源申请量为: 43.53252038433841
agent 4 在 1929 次迭代中的资源申请量为: 64.05934968488663
agent 5 在 1929 次迭代中的资源申请量为: 64.85934968488662
agent 6 在 1929 次迭代中的资源申请量为: 71.5852846533753
agent 7 在 1929 次迭代中的资源申请量为: 39.22383402809225
agent 8 在 1929 次迭代中的资源申请量为: 37.49264232668764
agent 9 在 1929 次迭代中的资源申请量为: 42.832520384338395
agent 10 在 1929 次迭代中的资源申请量为: 55.42714204105789
agent 11 在 1929 次迭代中的资源申请量为: 45.19080290835954
agent 12 在 1929 次迭代中的资源申请量为: 48.99018976891685
agent 13 在 1929 次迭代中的资源申请量为: 65.3593496848866
agent 14 在 1929 次迭代中的资源申请量为: 54.82714204105789
agent 15 在 1929 次迭代中的资源申请量为: 43.23252038433839
agent 16 在 1929 次迭代中的资源申请量为: 43.432520384338396
agent 17 在 1929 次迭代中的资源申请量为: 44.59080290835954
agent 18 在 1929 次迭代中的资源申请量为: 46.19080290835954
agent 19 在 1929 次迭代中的资源申请量为: 41.058491474097366
agent 20 在 1929 次迭代中的资源申请量为: 55.3271420410579
agent 21 在 1929 次迭代中的资源申请量为: 38.523834028092246
agent 22

1949 次迭代中的结果：
agent 1 在 1949 次迭代中的资源申请量为: 48.09018976891687
agent 2 在 1949 次迭代中的资源申请量为: 60.5877372111461
agent 3 在 1949 次迭代中的资源申请量为: 43.53252038433841
agent 4 在 1949 次迭代中的资源申请量为: 64.05934968488663
agent 5 在 1949 次迭代中的资源申请量为: 64.85934968488662
agent 6 在 1949 次迭代中的资源申请量为: 71.5852846533753
agent 7 在 1949 次迭代中的资源申请量为: 39.22383402809225
agent 8 在 1949 次迭代中的资源申请量为: 37.49264232668764
agent 9 在 1949 次迭代中的资源申请量为: 42.832520384338395
agent 10 在 1949 次迭代中的资源申请量为: 55.42714204105789
agent 11 在 1949 次迭代中的资源申请量为: 45.19080290835954
agent 12 在 1949 次迭代中的资源申请量为: 48.99018976891685
agent 13 在 1949 次迭代中的资源申请量为: 65.3593496848866
agent 14 在 1949 次迭代中的资源申请量为: 54.82714204105789
agent 15 在 1949 次迭代中的资源申请量为: 43.23252038433839
agent 16 在 1949 次迭代中的资源申请量为: 43.432520384338396
agent 17 在 1949 次迭代中的资源申请量为: 44.59080290835954
agent 18 在 1949 次迭代中的资源申请量为: 46.19080290835954
agent 19 在 1949 次迭代中的资源申请量为: 41.058491474097366
agent 20 在 1949 次迭代中的资源申请量为: 55.3271420410579
agent 21 在 1949 次迭代中的资源申请量为: 38.523834028092246
agent 22

1969 次迭代中的结果：
agent 1 在 1969 次迭代中的资源申请量为: 48.09018976891687
agent 2 在 1969 次迭代中的资源申请量为: 60.5877372111461
agent 3 在 1969 次迭代中的资源申请量为: 43.53252038433841
agent 4 在 1969 次迭代中的资源申请量为: 64.05934968488663
agent 5 在 1969 次迭代中的资源申请量为: 64.85934968488662
agent 6 在 1969 次迭代中的资源申请量为: 71.5852846533753
agent 7 在 1969 次迭代中的资源申请量为: 39.22383402809225
agent 8 在 1969 次迭代中的资源申请量为: 37.49264232668764
agent 9 在 1969 次迭代中的资源申请量为: 42.832520384338395
agent 10 在 1969 次迭代中的资源申请量为: 55.42714204105789
agent 11 在 1969 次迭代中的资源申请量为: 45.19080290835954
agent 12 在 1969 次迭代中的资源申请量为: 48.99018976891685
agent 13 在 1969 次迭代中的资源申请量为: 65.3593496848866
agent 14 在 1969 次迭代中的资源申请量为: 54.82714204105789
agent 15 在 1969 次迭代中的资源申请量为: 43.23252038433839
agent 16 在 1969 次迭代中的资源申请量为: 43.432520384338396
agent 17 在 1969 次迭代中的资源申请量为: 44.59080290835954
agent 18 在 1969 次迭代中的资源申请量为: 46.19080290835954
agent 19 在 1969 次迭代中的资源申请量为: 41.058491474097366
agent 20 在 1969 次迭代中的资源申请量为: 55.3271420410579
agent 21 在 1969 次迭代中的资源申请量为: 38.523834028092246
agent 22

1989 次迭代中的结果：
agent 1 在 1989 次迭代中的资源申请量为: 48.09018976891687
agent 2 在 1989 次迭代中的资源申请量为: 60.5877372111461
agent 3 在 1989 次迭代中的资源申请量为: 43.53252038433841
agent 4 在 1989 次迭代中的资源申请量为: 64.05934968488663
agent 5 在 1989 次迭代中的资源申请量为: 64.85934968488662
agent 6 在 1989 次迭代中的资源申请量为: 71.5852846533753
agent 7 在 1989 次迭代中的资源申请量为: 39.22383402809225
agent 8 在 1989 次迭代中的资源申请量为: 37.49264232668764
agent 9 在 1989 次迭代中的资源申请量为: 42.832520384338395
agent 10 在 1989 次迭代中的资源申请量为: 55.42714204105789
agent 11 在 1989 次迭代中的资源申请量为: 45.19080290835954
agent 12 在 1989 次迭代中的资源申请量为: 48.99018976891685
agent 13 在 1989 次迭代中的资源申请量为: 65.3593496848866
agent 14 在 1989 次迭代中的资源申请量为: 54.82714204105789
agent 15 在 1989 次迭代中的资源申请量为: 43.23252038433839
agent 16 在 1989 次迭代中的资源申请量为: 43.432520384338396
agent 17 在 1989 次迭代中的资源申请量为: 44.59080290835954
agent 18 在 1989 次迭代中的资源申请量为: 46.19080290835954
agent 19 在 1989 次迭代中的资源申请量为: 41.058491474097366
agent 20 在 1989 次迭代中的资源申请量为: 55.3271420410579
agent 21 在 1989 次迭代中的资源申请量为: 38.523834028092246
agent 22

In [6]:
np.savetxt('../data/03-robust_agent_resource_quantity_small_value_Gaussian_bias(80,10).txt',agent_resource_quantity,fmt='%0.8f')#用户资源量
np.savetxt('../data/03-robust_sum_cost_small_value_Gaussian_bias(80,10).txt',sum_cost,fmt='%0.8f')#用户成本总和 
np.savetxt('../data/03-robust_sum_resource_small_value_Gaussian_bias(80,10).txt',sum_resource,fmt='%0.8f')#用户资源总和 
np.savetxt('../data/03-robust_agent_gradient_small_value_Gaussian_bias(80,10).txt',agent_gradient,fmt='%0.8f')#用户迭代梯度 